In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile8 = pd.read_pickle("FFF_Train_Decile8.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile8.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile8, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150,200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 8')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 6s 2ms/step - loss: 0.2849 - acc: 0.8887 - val_loss: 0.2661 - val_acc: 0.8953
Epoch 39/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2838 - acc: 0.8953 - val_loss: 0.2648 - val_acc: 0.8966
Epoch 40/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2824 - acc: 0.8920 - val_loss: 0.2658 - val_acc: 0.8966
Epoch 41/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2800 - acc: 0.8979 - val_loss: 0.2656 - val_acc: 0.8953
Epoch 42/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2827 - acc: 0.8943 - val_loss: 0.2674 - val_acc: 0.8953
Epoch 43/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2806 - acc: 0.8969 - val_loss: 0.2663 - val_acc: 0.8953
Epoch 44/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2804 - acc: 0.8917 - val_loss: 0.2651 - val_acc: 0.8979

Epoch 00044: ReduceLROnPlateau reducing learning rate to 1e-

Epoch 98/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2705 - acc: 0.8979 - val_loss: 0.2635 - val_acc: 0.8979
Epoch 99/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2716 - acc: 0.8959 - val_loss: 0.2638 - val_acc: 0.8979
Epoch 100/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2749 - acc: 0.8969 - val_loss: 0.2643 - val_acc: 0.8940
Test accuracy: 0.8939790566554244
Train on 3055 samples, validate on 764 samples
Epoch 1/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.5829 - acc: 0.6622 - val_loss: 0.4331 - val_acc: 0.8599
Epoch 2/150
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4168 - acc: 0.8615 - val_loss: 0.3445 - val_acc: 0.8822
Epoch 3/150
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3518 - acc: 0.8763 - val_loss: 0.3113 - val_acc: 0.8914
Epoch 4/150
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3195 - acc: 0.8871 - val_l

Epoch 56/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2395 - acc: 0.9028 - val_loss: 0.2345 - val_acc: 0.9045
Epoch 57/150
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2359 - acc: 0.9018 - val_loss: 0.2343 - val_acc: 0.9045
Epoch 58/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2403 - acc: 0.9038 - val_loss: 0.2342 - val_acc: 0.9045
Epoch 59/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2364 - acc: 0.9064 - val_loss: 0.2343 - val_acc: 0.9058
Epoch 60/150
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2411 - acc: 0.9034 - val_loss: 0.2343 - val_acc: 0.9058
Epoch 61/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2376 - acc: 0.9047 - val_loss: 0.2343 - val_acc: 0.9045
Epoch 62/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2390 - acc: 0.9025 - val_loss: 0.2341 - val_acc: 0.9045
Epoch 63/150
3055/3055 [=================

Epoch 116/150
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2382 - acc: 0.9025 - val_loss: 0.2342 - val_acc: 0.9031
Epoch 117/150
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2334 - acc: 0.9070 - val_loss: 0.2344 - val_acc: 0.9018
Epoch 118/150
3055/3055 [==============================] - 8s 3ms/step - loss: 0.2344 - acc: 0.9047 - val_loss: 0.2347 - val_acc: 0.9045
Epoch 119/150
3055/3055 [==============================] - 8s 3ms/step - loss: 0.2418 - acc: 0.9061 - val_loss: 0.2337 - val_acc: 0.9031
Epoch 120/150
3055/3055 [==============================] - 11s 3ms/step - loss: 0.2396 - acc: 0.9028 - val_loss: 0.2342 - val_acc: 0.9018
Epoch 121/150
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2368 - acc: 0.9034 - val_loss: 0.2350 - val_acc: 0.9018
Epoch 122/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2382 - acc: 0.9067 - val_loss: 0.2342 - val_acc: 0.9045
Epoch 123/150
3055/3055 [===========

...

3055/3055 [==============================] - 5s 2ms/step - loss: 0.2871 - acc: 0.8907 - val_loss: 0.2650 - val_acc: 0.8914
Epoch 67/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2722 - acc: 0.8992 - val_loss: 0.2679 - val_acc: 0.8901
Epoch 68/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2775 - acc: 0.8969 - val_loss: 0.2661 - val_acc: 0.8901
Epoch 69/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2729 - acc: 0.8972 - val_loss: 0.2653 - val_acc: 0.8927
Epoch 70/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2742 - acc: 0.8982 - val_loss: 0.2705 - val_acc: 0.8914

Epoch 00070: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 71/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2789 - acc: 0.8979 - val_loss: 0.2658 - val_acc: 0.8914
Epoch 72/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2761 - acc: 0.8985 - val_loss: 0.2646 - val_acc: 0.8

3055/3055 [==============================] - 6s 2ms/step - loss: 0.2579 - acc: 0.8972 - val_loss: 0.2417 - val_acc: 0.9005
Epoch 26/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2514 - acc: 0.8989 - val_loss: 0.2500 - val_acc: 0.8927
Epoch 27/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2614 - acc: 0.9011 - val_loss: 0.2430 - val_acc: 0.8966
Epoch 28/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2559 - acc: 0.9002 - val_loss: 0.2911 - val_acc: 0.8901
Epoch 29/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2608 - acc: 0.8998 - val_loss: 0.2482 - val_acc: 0.9005
Epoch 30/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2538 - acc: 0.9005 - val_loss: 0.2502 - val_acc: 0.8979

Epoch 00030: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 31/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2521 - acc: 0.8982 - val_loss: 0.23

3055/3055 [==============================] - 6s 2ms/step - loss: 0.2393 - acc: 0.9028 - val_loss: 0.2322 - val_acc: 0.9045
Epoch 84/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2381 - acc: 0.9021 - val_loss: 0.2323 - val_acc: 0.9031
Epoch 85/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2393 - acc: 0.9005 - val_loss: 0.2322 - val_acc: 0.9045
Epoch 86/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2346 - acc: 0.9057 - val_loss: 0.2322 - val_acc: 0.9045
Epoch 87/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2375 - acc: 0.9047 - val_loss: 0.2322 - val_acc: 0.9045
Epoch 88/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2382 - acc: 0.9064 - val_loss: 0.2325 - val_acc: 0.9031
Epoch 89/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2383 - acc: 0.9077 - val_loss: 0.2327 - val_acc: 0.9045
Epoch 90/200
3055/3055 [==============================] - 6s 

3055/3055 [==============================] - 6s 2ms/step - loss: 0.2386 - acc: 0.9008 - val_loss: 0.2324 - val_acc: 0.9071
Epoch 144/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2389 - acc: 0.9021 - val_loss: 0.2322 - val_acc: 0.9058
Epoch 145/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2418 - acc: 0.9041 - val_loss: 0.2324 - val_acc: 0.9058
Epoch 146/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2384 - acc: 0.9057 - val_loss: 0.2325 - val_acc: 0.9071
Epoch 147/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2377 - acc: 0.9054 - val_loss: 0.2325 - val_acc: 0.9071
Epoch 148/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2378 - acc: 0.9025 - val_loss: 0.2324 - val_acc: 0.9058
Epoch 149/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2405 - acc: 0.9041 - val_loss: 0.2323 - val_acc: 0.9058
Epoch 150/200
3055/3055 [==============================

3055/3055 [==============================] - 6s 2ms/step - loss: 0.5078 - acc: 0.7378 - val_loss: 0.4333 - val_acc: 0.8796
Epoch 3/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.4331 - acc: 0.8452 - val_loss: 0.3726 - val_acc: 0.8796
Epoch 4/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3688 - acc: 0.8632 - val_loss: 0.3087 - val_acc: 0.8848
Epoch 5/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3345 - acc: 0.8773 - val_loss: 0.2967 - val_acc: 0.8940
Epoch 6/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3207 - acc: 0.8825 - val_loss: 0.2854 - val_acc: 0.8927
Epoch 7/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3058 - acc: 0.8884 - val_loss: 0.2771 - val_acc: 0.8979
Epoch 8/50
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3035 - acc: 0.8887 - val_loss: 0.2765 - val_acc: 0.8966
Epoch 9/50
3055/3055 [==============================] - 6s 2ms/step - los

In [ ]:
0.93484

In [ ]:
#0.8783